<a href="https://colab.research.google.com/github/AyushiBhansali/Book-Recommendation-System-ML/blob/master/bookrecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163494 sha256=8341f33affa9d83e4f7dd8da078aecdb47b2dfb73b803b21aa22d55760e704e0
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy


In [10]:
from google.colab import files
uploaded = files.upload()


Saving Books.csv to Books.csv
Saving Ratings.csv to Ratings.csv
Saving recsys_taxonomy2.png to recsys_taxonomy2.png
Saving Users.csv to Users.csv


In [20]:
# Check the names of the uploaded files
print(uploaded.keys())

# Load the Books, Ratings, and Users data
books_df = pd.read_csv(io.BytesIO(uploaded['Books.csv']))
ratings_df = pd.read_csv(io.BytesIO(uploaded['Ratings.csv']))
users_df = pd.read_csv(io.BytesIO(uploaded['Users.csv']))

# Explore the structure of the data (optional)
print("Books Data:")
print(books_df.head())

print("\nRatings Data:")
print(ratings_df.head())

print("\nUsers Data:")
print(users_df.head())

# Merge Ratings and Books data
merged_df = pd.merge(ratings_df, books_df, on='ISBN', how='inner')

# Surprise requires a specific format (user, item, rating)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(merged_df[['User-ID', 'ISBN', 'Book-Rating']], reader)

# Split the data
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Train the model
model = SVD()
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Evaluate the model
accuracy.rmse(predictions)


dict_keys(['Books.csv', 'Ratings.csv', 'recsys_taxonomy2.png', 'Users.csv'])
Books Data:
         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-

3.511885618247029

In [21]:
# Example: Recommend books for a specific user
def recommend_books(user_id, n=5):
    # Get a list of all book IDs
    all_books = merged_df['ISBN'].unique()

    # Remove books already rated by the user
    rated_books = merged_df[merged_df['User-ID'] == user_id]['ISBN'].tolist()
    unrated_books = list(set(all_books) - set(rated_books))

    # Make predictions for unrated books
    predictions = [model.predict(user_id, book_id).est for book_id in unrated_books]

    # Get indices of top n predictions
    top_indices = sorted(range(len(predictions)), key=lambda i: predictions[i], reverse=True)[:n]

    # Get the corresponding ISBNs
    top_books = [unrated_books[i] for i in top_indices]

    # Return the top recommended books
    return books_df[books_df['ISBN'].isin(top_books)]

# Example: Recommend top 5 books for User-ID 123
recommended_books = recommend_books(123, n=5)
print(recommended_books[['ISBN', 'Book-Title']])


             ISBN                                         Book-Title
2809   0590353403     Harry Potter and the Sorcerer's Stone (Book 1)
2812   0380815931            In the Beginning...was the Command Line
5685   2253152846                             Metaphysique Des Tubes
5736   1558506462  Small Miracles: Extraordinary Coincidences fro...
19392  0380978407  Lamb: The Gospel According to Biff, Christ's C...
